In [6]:
import heapq
import math
import time
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

class NPuzzleSolver:
    def __init__(self, initial_state, goal_state, size):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.size = size
        self.goal_positions = {val: (i // size, i % size) for i, val in enumerate(goal_state)}

    def get_neighbors(self, state):
        size = int(math.sqrt(len(state)))
        blank_idx = state.index(0)
        blank_row, blank_col = blank_idx // size, blank_idx % size

        neighbors = []
        moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]

        for move in moves:
            new_row, new_col = blank_row + move[0], blank_col + move[1]
            if 0 <= new_row < size and 0 <= new_col < size:
                new_idx = new_row * size + new_col
                new_state = list(state)
                new_state[blank_idx], new_state[new_idx] = new_state[new_idx], new_state[blank_idx]
                neighbors.append(tuple(new_state))

        return neighbors

    def heuristic_misplaced_tiles(self, state):
        return sum(1 for i in range(len(state)) if state[i] != self.goal_state[i] and state[i] != 0)

    def heuristic_manhattan_distance(self, state):
        total_distance = 0
        for i in range(len(state)):
            if state[i] != 0:
                current_row, current_col = i // self.size, i % self.size
                goal_row, goal_col = self.goal_positions[state[i]]
                total_distance += abs(current_row - goal_row) + abs(current_col - goal_col)
        return total_distance

    def heuristic_euclidean_distance(self, state):
        total_distance = 0
        for i in range(len(state)):
            if state[i] != 0:
                current_row, current_col = i // self.size, i % self.size
                goal_row, goal_col = self.goal_positions[state[i]]
                total_distance += math.sqrt((current_row - goal_row) ** 2 + (current_col - goal_col) ** 2)
        return total_distance

    def heuristic_linear_conflict(self, state):
        linear_conflict = self.heuristic_manhattan_distance(state)
        for row in range(self.size):
            row_conflict = []
            for col in range(self.size):
                tile = state[row * self.size + col]
                if tile != 0 and self.goal_positions[tile][0] == row:
                    row_conflict.append(tile)
            linear_conflict += self.calculate_conflicts(row_conflict)
        for col in range(self.size):
            col_conflict = []
            for row in range(self.size):
                tile = state[row * self.size + col]
                if tile != 0 and self.goal_positions[tile][1] == col:
                    col_conflict.append(tile)
            linear_conflict += self.calculate_conflicts(col_conflict)
        return linear_conflict

    def calculate_conflicts(self, tiles):
        conflict_count = 0
        for i in range(len(tiles)):
            for j in range(i + 1, len(tiles)):
                if tiles[i] > tiles[j]:
                    conflict_count += 2
        return conflict_count

    def best_first_search(self, heuristic):
        frontier = []
        heapq.heappush(frontier, (0, self.initial_state))
        explored = set()
        parent_map = {self.initial_state: None}
        cost_map = {self.initial_state: 0}

        while frontier:
            _, current_state = heapq.heappop(frontier)

            if current_state == self.goal_state:
                return self.reconstruct_path(parent_map)

            explored.add(current_state)
            for neighbor in self.get_neighbors(current_state):
                if neighbor in explored:
                    continue
                new_cost = cost_map[current_state] + 1
                if neighbor not in cost_map or new_cost < cost_map[neighbor]:
                    cost_map[neighbor] = new_cost
                    priority = new_cost + heuristic(neighbor)
                    heapq.heappush(frontier, (priority, neighbor))
                    parent_map[neighbor] = current_state

        return None

    def reconstruct_path(self, parent_map):
        state = self.goal_state
        path = []
        while state:
            path.append(state)
            state = parent_map[state]
        return path[::-1]

class NPuzzleGUI:
    def __init__(self, solver, size):
        self.solver = solver
        self.size = size
        self.initial_state = list(solver.initial_state)
        self.goal_state = solver.goal_state
        self.tiles = []
        self.create_ui()

    def create_ui(self):
        self.tiles = [widgets.Button(description=str(val) if val != 0 else '', layout=widgets.Layout(width='50px', height='50px'))
                      for val in self.initial_state]
        for tile in self.tiles:
            tile.on_click(self.on_tile_click)

        grid = widgets.GridBox(self.tiles, layout=widgets.Layout(grid_template_columns=' '.join(['50px'] * self.size)))

        self.heuristic_selector = widgets.Dropdown(
            options=['Misplaced Tiles', 'Manhattan Distance', 'Euclidean Distance', 'Linear Conflict'],
            value='Manhattan Distance',
            description='Heuristic:',
            style={'description_width': 'initial'}
        )

        self.solve_button = widgets.Button(description='Solve', button_style='success', layout=widgets.Layout(width='100px'))
        self.solve_button.on_click(self.solve_puzzle)

        self.output = widgets.Output()

        self.message_label = widgets.HTML(value="<b>Select Heuristic and Solve the Puzzle!</b>", layout=widgets.Layout(margin='10px 0px 10px 0px'))

        display(widgets.VBox([self.message_label, self.heuristic_selector, self.solve_button, grid, self.output]))

    def on_tile_click(self, button):
        idx = self.tiles.index(button)
        blank_idx = self.tiles.index([t for t in self.tiles if t.description == ''][0])
        if abs(idx - blank_idx) == 1 or abs(idx - blank_idx) == self.size:
            self.tiles[blank_idx].description = button.description
            button.description = ''

    def solve_puzzle(self, button):
        with self.output:
            clear_output()
            heuristic_map = {
                'Misplaced Tiles': self.solver.heuristic_misplaced_tiles,
                'Manhattan Distance': self.solver.heuristic_manhattan_distance,
                'Euclidean Distance': self.solver.heuristic_euclidean_distance,
                'Linear Conflict': self.solver.heuristic_linear_conflict
            }
            selected_heuristic = heuristic_map[self.heuristic_selector.value]
            solution_path = self.solver.best_first_search(selected_heuristic)

            if solution_path:
                self.message_label.value = "<b>Solving...</b>"
                self.animate_solution(solution_path)
                self.message_label.value = "<b>Puzzle Solved!</b>"
            else:
                self.message_label.value = "<b>No solution found!</b>"

    def animate_solution(self, solution_path):
        for state in solution_path:
            self.update_tiles(state)
            display(self.get_grid())
            time.sleep(0.5)

    def update_tiles(self, state):
        for i, val in enumerate(state):
            self.tiles[i].description = str(val) if val != 0 else ''

    def get_grid(self):
        return widgets.GridBox(self.tiles, layout=widgets.Layout(grid_template_columns=' '.join(['50px'] * self.size)))

initial_state = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 13, 14, 15, 12)
goal_state = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)

solver = NPuzzleSolver(initial_state, goal_state, size=4)
gui = NPuzzleGUI(solver, size=4)
